In [2]:
# 1. Imports 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

# Scikit-learn modeling and validation
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# --- FIX 1: Split calibration_curve from metrics ---
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             confusion_matrix, classification_report, roc_curve, precision_recall_curve,
                             brier_score_loss)
from sklearn.calibration import calibration_curve # Moved to its correct module

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, backend as K

# --- FIX 2: Modern Keras Tuner naming ---
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch

# Model Interpretability
import shap
import lime
import lime.lime_tabular

warnings.filterwarnings('ignore')

# 2. Data Loading & Preprocessing

# Load data
df = pd.read_csv('heart_disease_eda_advanced.csv')

# Separate features and target
X = df.drop('target', axis=1)
y = df['target'].astype(int)

# --- FIX 3: Robust Categorical Encoding ---
# This identifies both 'object' (strings like '60+') and 'category' types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    # Using factorize handles strings better than .cat.codes if types aren't explicitly 'category'
    X[col] = pd.factorize(X[col])[0]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Feature Selection & Scaling

# Reuse saved artifacts from previous steps
try:
    top_features = joblib.load('top_features.pkl')
    scaler_nn = joblib.load('scaler_nn.pkl')
    
    # Filter to selected features
    X_train_fs = X_train[top_features]
    X_test_fs = X_test[top_features]

    # Apply scaling
    X_train_scaled = scaler_nn.transform(X_train_fs)
    X_test_scaled = scaler_nn.transform(X_test_fs)
    
    print("Preprocessed data ready: Used", len(top_features), "top features.")
    
except FileNotFoundError:
    print("Error: .pkl files not found. Ensure you saved 'top_features.pkl' and 'scaler_nn.pkl' in the previous notebook.")

Preprocessed data ready: Used 10 top features.


In [3]:
# 2. Hyperparameter Tuning for TensorFlow Models with Keras Tuner (FIXED)

import keras_tuner as kt # Use the modern library name
from tensorflow.keras import backend as K

# Clear any previous sessions to reset internal metric counters
K.clear_session()

# Define model building function for tuner
def build_model(hp):
    model = keras.Sequential()
    
    # Use the actual shape of your processed training data
    model.add(layers.Input(shape=(X_train_fs.shape[1],)))
    
    # Hyperparameters to tune
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    hp_layers = hp.Int('layers', min_value=1, max_value=3)
    hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
    hp_lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    hp_l2 = hp.Float('l2', min_value=1e-5, max_value=1e-3, sampling='log')
    
    # Build hidden layers
    for i in range(hp_layers):
        model.add(layers.Dense(
            units=hp_units, 
            activation='relu',
            kernel_regularizer=regularizers.l2(hp_l2)
        ))
        model.add(layers.Dropout(rate=hp_dropout))
        model.add(layers.BatchNormalization())
        
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # FIX: Explicitly name the metric 'auc' so the tuner can find 'val_auc' reliably
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_lr),
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC(name='auc')] 
    )
    return model

# Initialize the Tuner
tuner = kt.RandomSearch(
    build_model,
    # FIX: Use the explicit Objective object
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=10,
    executions_per_trial=1, # Set to 1 for faster debugging
    directory='tuner_results',
    project_name='heart_disease_v2', # Using a new name to avoid old log conflicts
    overwrite=True # FIX: Automatically clear previous failed attempts/cache
)

# Start searching
# Note: verbose=1 is better for debugging so you can see if an epoch fails
tuner.search(
    X_train_scaled, 
    y_train, 
    epochs=30, 
    validation_split=0.2, 
    batch_size=16, 
    verbose=1 
)

# --- Retrieve Results ---

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n" + "="*30)
print("BEST HYPERPARAMETERS FOUND:")
print(f"Units: {best_hps.get('units')}")
print(f"Layers: {best_hps.get('layers')}")
print(f"Dropout: {best_hps.get('dropout'):.2f}")
print(f"Learning Rate: {best_hps.get('learning_rate'):.4f}")
print(f"L2 Regularization: {best_hps.get('l2'):.5f}")
print("="*30 + "\n")

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

Traceback (most recent call last):
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        results,
        ^^^^^^^^
        self.oracle.objective,
        ^^^^^^^^^^^^^^^^^^^^^^
    ),
    ^
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        results,
        ^^^^^^^^
        self.oracle.objective,
        ^^^^^^^^^^^^^^^^^^^^^^
    ),
    ^
  File "C:\Users\phill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras_tuner\src\engine\tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^